# Remote Task Creation via ATSVC named pipe
Detects remote task creation via at.exe or API interacting with ATSVC namedpipe

## Rule Content
```
- title: Remote Task Creation via ATSVC named pipe
  id: f6de6525-4509-495a-8a82-1f8b0ed73a00
  description: Detects remote task creation via at.exe or API interacting with ATSVC
    namedpipe
  author: Samir Bousseaden
  references:
  - https://blog.menasec.net/2019/03/threat-hunting-25-scheduled-tasks-for.html
  tags:
  - attack.lateral_movement
  - attack.persistence
  - attack.t1053
  - car.2013-05-004
  - car.2015-04-001
  logsource:
    product: windows
    service: security
    description: The advanced audit policy setting "Object Access > Audit Detailed
      File Share" must be configured for Success/Failure
    category: null
  detection:
    selection:
      EventID: 5145
      ShareName: \\*\IPC$
      RelativeTargetName: atsvc
      Accesses: '*WriteData*'
    condition: selection
  falsepositives:
  - pentesting
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-security-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"5145" AND share_name.keyword:\\*\\IPC$ AND share_relative_target_name:"atsvc" AND Accesses.keyword:*WriteData*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()